In [ ]:
# default_exp core

# attractor

The `attractor` module contains classes that generate roulette curves ("spirographs"), strange attractors, and other dynamical systems. It is designed with generalization and modularity in mind, so the systems can be simulated in an arbitrary number of dimensions and allow versatile rules for generation. The main goals of this project are:

 - Intuitive, object-oriented methods for simulating, analyzing, and aggregating statistics for certain classes of dynamical systems
 - Customizable and efficient visualization of both individual systems and classes of systems
 - Tools for painlessly starting and stopping simulations/renderers, caching/storing/loading results, and reusing or tweaking partial results
 - Panels for interactive exploration of attractor systems built on real-time adaptive simulations & rendering

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

# %matplotlib widget
import IPython
# IPython.get_ipython().run_line_magic('matplotlib', 'widget')
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import math
import random
import time
from scipy import signal, misc

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

In [ ]:
# todo: multipoint systems
# todo: chaos analysis

In [ ]:
#export

class Attractor:
    def __init__(self):
        pass

In [ ]:
#export

class IteratedFunctionSystem(Attractor):
    def __init__(self):
        pass

In [ ]:
#export

@nb.njit
def rotation_matrix(a):
    sin = np.sin(a)
    cos = np.cos(a)
    R = [
        [cos, -sin],
        [sin, cos]
    ]
    return np.array(R)

In [ ]:
#export

@nb.njit
def rotate(a, b, t):
#         if type(t) in [int, float]:
#         if isinstance(t, (int, float)):
    t = rotation_matrix(t)

#         return (rMatrix @ (a - b)) + b
    return (t @ (a - b)) + b

In [ ]:
#export

@nb.njit
def simulate_accelerated(speeds, pivots, center, angles, start, points, steps=100, clip=True):
    """
    Efficiently simulate a system of rotating segments joined by pivots using Numba to JIT-compile operations on NumPy arrays.
    
    - `steps`: integer >=1; the number of timesteps to simulate
    - `clip`: `boolean`; whether to limit the maximum angle of each section (if `True`, the values will wrap around to 0; defaults to `True`)
    """
#     todo: reuse code across this function and the other simulate function

#     assert type(steps) is int
#     assert isinstance(steps, int)
#     assert steps >= 1
#     assert len(speeds) > 0
#     assert len(pivots) > 0
    
    rMatrices = []
    for s in speeds:
        rMatrices.append(rotation_matrix(s))
    num_pivots = len(pivots)
    for s in range(steps):
        for l in list(range(num_pivots)):
            rMatrix = rMatrices[l]
            offsets = center if l == 0 else pivots[l-1]
            angles[l:] += speeds[l]
            if clip:
                angles[l:] %= 2 * math.pi
        prev = rotate(start[0], center, angles[0])
        for p in range(1, num_pivots):
            pivots[p] = rotate(start[p], center, angles[p]) + prev
            prev = pivots[p]
#         if self.live_rendering:
#             self.draw_point(self.pivots[-1].copy(), 'pixel')
#         else:
        points = np.append(points, np.expand_dims(pivots[-1], axis=0), axis=0)
    return points
# dynamic wrappers for Numba functions

In [ ]:
doc(simulate_accelerated)

simulate_accelerated [source] simulate_accelerated ( speeds , pivots , center , angles , start , points , steps = 100 , clip = True ) 
 
 Efficiently simulate a system of rotating segments joined by pivots using Numba to JIT-compile operations on NumPy arrays. 
 
 steps : integer >=1; the number of timesteps to simulate 
 clip : boolean ; whether to limit the maximum angle of each section (if True , the values will wrap around to 0; defaults to True )

In [ ]:
#export

# @nb.jit
def line(start, stop, bg, width=1., quality=5.):
#     if bg is None:
#         bg = np.zeros((50, 50))
#     start = np.array(start, dtype=float)
#     stop = np.array(stop, dtype=float)
    assert quality >= 1
    assert width > 0
    
    quality *= np.linalg.norm(np.subtract(stop, start))
    lin = np.linspace(0, 1, round(quality))
#     coords = np.stack([[np.interp(lin, [0,1], [0, p[i]]) for p in [start, stop]] for i in range(0, 2)]).T
#     j = []
#     for i in range(0, 2):
#         j.append(np.interp(lin, [0,1], [start[i], stop[i]]))
    j = [np.interp(lin, [0,1], [start[i], stop[i]]) for i in [0, 1]]
    coords = np.stack(j).T
    for pos in coords:
        x, y = pos#.astype(int)
        w = width/2
        bg[round(x-w):round(x+w), round(y-w):round(y+w)] += 1
    return bg

In [ ]:
im = line(
    np.array([5., 10.]),
    np.array([40., 40.]),
    np.zeros((50, 50)),
    width=1
)
# print(im.max())
plt.close('all')
fig, ax = plt.subplots()
ax.imshow(im, interpolation='none')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#export

# @nb.jit
class RouletteCurve(Attractor):
    def __init__(self, center=[0, 0], num_sections=4, lengths=None, speeds=None, random_distribution='uniform'):
        """
        Create a new `RouletteCurve` object. This subclasses `Attractor` and describes a process where one or more line segments, connected end-to-end, rotate continuously about their pivots/endpoints. The length of each line segment and the speed at which it rotates are adjustable parameters.
        """
        super().__init__()
        self.random_distribution = getattr(np.random, random_distribution)
        self.rd = self.random_distribution
        assert center
        self.center = np.array(center, dtype=float)
        self.rank = self.center.size
#         use rank?
        if lengths is None:
            assert num_sections
            lengths = np.random.uniform(-2, 2, num_sections)
        if speeds is None:
            assert num_sections
            speeds = np.random.normal(0, 2, num_sections)
        self.lengths = RouletteCurve.randomize_list(lengths).astype(float)
        self.speeds = RouletteCurve.randomize_list(speeds).astype(float)
#         self.angles = np.random.normal(0, 2*math.pi, num_sections).astype(float)
        self.angles = np.zeros(num_sections, dtype=float)
        self.angles_ = []
        self.start = center + np.array([[0, sum(self.lengths[:i])] for i in range(1, len(self.lengths)+1)])
        self.start = self.start.astype(float)
        self.pivots = self.start.copy()
        self.pivots_ = []
#         self.points = []
        self.points = np.zeros([1, 2])
        self.canvas = np.zeros([100, 100])
        self.position = 0
        
        self.zoom = 10
        self.offset = 0
        self.N = 0
        self.live_rendering = False
        
#         See colormaps by category at https://matplotlib.org/stable/tutorials/colors/colormaps.html; used here are the "sequential" cmaps and a few others
        self.cmaps = ['inferno', 'plasma', 'rainbow', 'hot', 'cool', 'autumn', 'winter', 'summer'] + [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
    
    def clear(self):
        """
        Remove all generated points from this `Attractor`
        """
        self.points = np.zeros([1, 2])
        return self
    
    def get_state(self):
        """
        Internal/helper function; gets current values of this instance's speeds, pivots, angles, etc. as a dictionary (mainly for use in typed functions like simulate_accelerated)
        """
        return dict(
            speeds=self.speeds,
            pivots=self.pivots,
            center=self.center,
            angles=self.angles,
            start=self.start,
            points=self.points
        )
    
    def simulate_accelerated(self, steps, duration=None):
        """
        A faster (but less dynamic) version of `simulate` based on Numba. The actual computation is offloaded to a simplified, strongly typed global function. This method is several times faster than `simulate`, especially for more complex `Attractor`s.
        """
        assert steps or duration
        start_time = time.time()
        sim_args = dict(
            **self.get_state(),
            steps=steps,
            clip=True
        )
        if duration:
            while (time.time() - start_time) < duration:
                self.start = self.pivots.copy()
                new_points = simulate_accelerated(**sim_args)
                self.points = np.append(self.points, new_points.copy(), axis=0)
#                 why does refreshing these change the pattern? (m=2)
                sim_args = dict(**self.get_state(), steps=steps, clip=True)
        else:
            self.points = simulate_accelerated(**sim_args)
            
        return self
    
#     @nb.jit#(forceobj=True)
    def simulate(self, steps=None, render_each=None, render_settings={}, clip=True, duration=None, timecheck_frequency=100, live_rendering=True):
        """
        Simulate the system by calculating the position of each point from data about the preceding points, and updating the internal state accordingly.
        
        - `steps`: (optional) integer >=1; the number of timesteps to simulate
        - `render_each`: (optional) integer >=1 representing how many steps to run before re-rendering the simulation result
        - `render_settings`: (optional) a `dictionary` that will be passed to `render` if using `render_each`
        - `clip`: `boolean`; whether to limit the maximum angle of each section (if `True`, the values will wrap around to 0; defaults to `True`)
        - `duration`: `float` or `int` >0; the maximum length of time, in seconds, to run the simulation for; if `steps` is not provided, the simulation will run until this amount of time has elapsed
        - `timecheck_frequency`: Not yet documented
        - `live_rendering`: Not yet documented
        """
        
        self.live_rendering = live_rendering
        start_time = time.time()
        rMatrices = []
        assert self.speeds
        for s in self.speeds:
#             theta = 1 * self.speeds[l]
            rMatrices.append(rotation_matrix(s))
        print(duration)
#         for s in range(steps):
        assert steps or duration, 'Either the number of steps to simulate or the length of time to run the simulation for must be provided.'
        if duration:
            assert duration > 0
            assert isinstance(duration, (int, float))
        s = 0
        while (s < steps if steps else True):
#             last = self.pivots.copy()
#             for l in list(range(len(self.pivots)))[::-1]:
            gamma = 0
            num_pivots = len(self.pivots)
            for l in list(range(num_pivots)):
#             theta = 1 * self.speeds
                rMatrix = rMatrices[l]
#                 rMatrix = np.array(rMatrix)#.swapaxes(0,2)
                offsets = self.center if l == 0 else self.pivots[l-1]#.copy() #?
#                 for f in list(range(l, num_pivots)):
    #                 print(s, rMatrix)
        #             print(self.pivots[:-1].shape)
    #                 offsets = np.concatenate([self.center[np.newaxis,...], self.pivots[:-1]], axis=0)
    #                 offsets=np.array(0)
    #                 len(self.pivots)-1
    #                 print(offsets)
        #             print(offsets.shape, rMatrix.shape, self.pivots.shape)
    #                 self.pivots[l] = (last[l] - offsets) @ rMatrix + offsets

    #                 func of t?
    #                 delta = (last[l] - offsets) @ rMatrix + offsets
#                     delta = (rMatrix @ (self.pivots[f] - offsets)) + offsets# + gamma
    #                 print(delta)
#                     gamma += delta
#                     self.pivots[f] = delta

                self.angles[l:] += self.speeds[l]
                if clip:
                    self.angles[l:] %= 2 * math.pi
#                 self.angles_.append(self.angles.copy())

    #             self.points.append(np.clip(self.pivots[-1], 0, np.array(self.canvas.shape)))
#     sequencemethod

            prev = rotate(self.start[0], self.center, self.angles[0])
            for p in range(1, num_pivots):
#                 print(p, prev)
                self.pivots[p] = rotate(self.start[p], self.center, self.angles[p]) + prev
                prev = self.pivots[p]
            
            if self.live_rendering:
                self.draw_point(self.pivots[-1].copy(), mode='pixel')
            else:
#                 self.pivots_.append(self.pivots.copy())
#                 self.points.append(self.pivots[-1].copy())
#                 self.points = np.concatenate([self.points, self.pivots[-1].copy()[np.newaxis, ...]], axis=0)
                self.points = np.append(self.points, self.pivots[-1].copy()[np.newaxis, ...], axis=0)
            
            if render_each is not None:
                assert isinstance(render_each, int)
                assert render_each >= 1
                if s % render_each == 0:
                    assert render_settings is none or isinstance(render_settings, dict)
                    self.render(**render_settings)
        
            if (duration is not None) and s % timecheck_frequency == 0:
                elapsed = time.time() - start_time
                if elapsed > duration:
#                     make a function for this?
                    comp = f'; {round(s / steps * 100, 3)}% complete)' if steps else ''
                    g = 'Terminating simulation early' if steps else 'Ending simulation'
                    print(f'{g} after {s+1} steps ({round(elapsed, 3)} seconds elapsed)' + comp)
                    break
            s += 1
        self.N = s
        print(f'Finished simulating {steps or s+1} steps in {round(elapsed, 3)} seconds')
        return self
    
    def transform_point(self, p):
        p = p.astype(float)
        p *= self.zoom
#         if recenter:
        p += self.offset
        p = np.clip(p, 0, np.array(self.canvas.shape)-1)
        assert isinstance(p, np.ndarray)
        return p
    
    def draw_point(self, p, mode, blending='add', brush=None):
        if type(p) is int:
            prev = self.points[p-1]
            prev = self.transform_point(prev)
            p = self.points[p]
        else:
            assert isinstance(p, (np.ndarray, list, tuple))
        p = self.transform_point(p)
        w, h = self.canvas.shape
        x, y = p.astype(int)
        if mode == 'pixel':
            if blending == 'set':
                self.canvas[x, y] = 1
            elif blending == 'add':
                self.canvas[x, y] += 1
#                 TODO: add multiply mode
            else:
                raise ValueError
        elif mode in ['dist', 'brush']:
            assert isinstance(brush, np.ndarray)
            x, y = np.clip(x, 5, w-6), np.clip(y, 5, h-6)
            self.canvas[x-2:x+3, y-2:y+3] += brush
        elif mode in ['line']:
            assert prev is not None
            self.canvas = line(prev, p, self.canvas)
        else:
            raise ValueError
        return self
    
#     @nb.jit(forceobj=True)
    def render(self, discard=False, clip=False, axis=None, recenter=True, zoom=None, mode='line', blending='add', hist_args={}, cmap='random', point_value=1, falloff=3, **kwargs):
        """
        Render an image from the list of points stored in the class instance.
        
        - `discard`: `boolean`; if `True`, clear this `Attractor`'s points after rendering to free up memory
        - `axis`: A Matplotlib axis to render the finished image to (if one is not provided, it will be created)
        - `zoom`: A scaling factor by which to resize points relative to the `center` before rendering
        - `mode`: `str`, one of `pixel`, `dist`/`brush`, `line`, or `hist`
            -     `pixel`: Convert each point coordinate to the (integer) coordinate of the nearest pixel
            -     `dist`: Generate a "brush" based on a distance function relative to the point coordinates, then (for each point) paint this over a region of the canvas centered on that point
            -     `line`: Draw a line from the last rendered point to the current one (helpful for reducing the total number of points that must be rendered)
            -     `hist`: Generate a 2D histogram using NumPy and display this (similar to `pixel`)
        - `blending`: `str`, one of `set`, `add`, or `mul`; how the new pixel value (while rendering a point) should be combined with the current one
        - `cmap`: `random` or a Matplotlib colormap; the colormap to pass to `imshow` - if `random`, one will be selected from the sequential colormaps listed in `RouletteCurve().cmaps`
        - `point_value`
        - `**kwargs`
        """
        
        assert mode in ['hist', 'dist', 'brush', 'pixel', 'line']
        assert len(self.points) >= 1
        self.points = self.points[1:]
        cshape = np.array(self.canvas.shape)
        self.offset = cshape / 2
        if not self.live_rendering:# and (self.points):
            if zoom is None:
                zoom = np.min(cshape / np.max(np.abs(self.points), axis=0)) * 0.5
#             else:
            assert isinstance(zoom, (int, float))
            self.zoom = zoom
#         for p in self.points.copy():
        if mode == 'dist':
#             grid = np.stack(np.meshgrid([np.arange(5.)]*2))
            grid = np.mgrid[0:5, 0:5]
            assert isinstance(falloff, int) and falloff >= 1
            brush = 1 / np.linalg.norm(grid - 2.5, axis=0, ord=falloff)
            print(brush)
        
        if not self.live_rendering:
            if mode == 'hist':
                assert hist_args is None or isinstance(hist_args, dict)
                self.canvas = np.histogram2d(*np.array(self.points).T, **hist_args)[0]
                assert isinstance(point_value, (int, float))
#                 self.canvas *= point_value
                if blending == 'set':
    #                 self.canvas = np.power(self.canvas, 0)
                    self.canvas[self.canvas != 0] = point_value
                elif blending == 'mul':
                    self.canvas = point_value ** self.canvas
                else:
                    assert blending == 'add'
            else:
                if mode in ['line']:
                    for i in range(1, len(self.points)):
#                         self.canvas = line(self.points[i-1], self.points[i], self.canvas)
                        self.draw_point(i, mode=mode)
                else:
                    for i, p in enumerate(map(np.copy, self.points)):
            #             for j, p in enumerate(px):
                        if mode in ['dist', 'brush']:
                            self.draw_point(p, mode=mode, brush=brush)
                        else:
                            self.draw_point(p, mode=mode)
            #                 self.canvas[x, y] = j+1
            #         plt.style.use('fivethirtyeight')
        
        
#         pendulums
        if cmap == 'random':
            cmap = random.choice(self.cmaps)
        else:
            assert (isinstance(cmap, str) and cmap in self.cmaps) or (isinstance(plt.cm.colors.Colormap) or cmap in plt.colormaps())
            
        plot_args = dict(X=np.flip(self.canvas.T, axis=0), interpolation='none', cmap=cmap, **kwargs)
        if axis:
            P = axis.imshow(**plot_args)
        else:
            plt.style.use('classic')
            P = plt.imshow(**plot_args)
            plt.grid('off')
        
        if discard:
            self.clear()
        if clip:
            assert len(clip) == 2
            assert isinstance(clip, (np.ndarray, list, tuple))
            self.canvas = np.clip(self.canvas, *clip)
        
        return P
#         return self
    
    @staticmethod
    def randomize_list(L):
        assert isinstance(L, (np.ndarray, list, tuple))
        return np.array([self.rd(*x) if type(x) in [list, tuple] else x for x in L])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.cool()
plt.cm.colors.Colormap('cool')

In [ ]:
m = 2
plt.close('all')
R = RouletteCurve(num_sections=m)
# R = RouletteCurve(lengths=[10]*m, speeds=[1]*m)
# print(R.pivots, R.points)
# print(R.start, R.lengths)
# R.simulate(live_rendering=False, duration=5).render(mode='hist', blending='add', hist_args=dict(bins=100)
#                           cmap=random.choice(plt.colormaps())
#                          )
# todo: estimate completion time
# print(R.start, R.lengths)

R.simulate_accelerated(steps=100).render(discard=True, mode='line', blending='add', point_value=1, hist_args=dict(bins=150))
print(R.pivots, R.center)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 0.         -1.09736288]
 [-0.06505168  1.22416644]] [0. 0.]


In [ ]:
# doc(RouletteCurve.simulate)
doc(RouletteCurve.render)

RouletteCurve.render [source] RouletteCurve.render ( discard = False , clip = False , axis = None , recenter = True , zoom = None , mode = 'line' , blending = 'add' , hist_args = {} , cmap = 'random' , point_value = 1 , ** kwargs ) 
 
 Render an image from the list of points stored in the class instance. 
 
 discard : boolean ; if True , clear this Attractor 's points after rendering to free up memory 
 axis : A Matplotlib axis to render the finished image to (if one is not provided, it will be created) 
 zoom : A scaling factor by which to resize points relative to the center before rendering 
 mode : str , one of pixel , dist / brush , line , or hist 
 pixel : Convert each point coordinate to the (integer) coordinate of the nearest pixel 
 dist : Generate a "brush" based on a distance function relative to the point coordinates, then (for each point) paint this over a region of the canvas centered on that point 
 line : Draw a line from the last rendered point to the current one (helpful for reducing the total number of points that must be rendered) 
 hist : Generate a 2D histogram using NumPy and display this (similar to pixel ) 
 
 
 blending : str , one of set , add , or mul ; how the new pixel value (while rendering a point) should be combined with the current one 
 cmap : random or a Matplotlib colormap; the colormap to pass to imshow - if random , one will be selected from the sequential colormaps listed in RouletteCurve().cmaps 
 point_value 
 **kwargs

In [ ]:
np.interp([0.1, 0.2], [0,1], [0,5])

array([0.5, 1. ])

In [ ]:
R2 = RouletteCurve(num_sections=3).simulate_accelerated(steps=1000, duration=1)
R2.render(mode='hist', blending='add', point_value=1, hist_args=dict(bins=200))
# why is this so fast?
print(len(R2.points))

255255


In [ ]:
1 ** np.array([5, 6, 0])

array([1, 1, 1], dtype=int32)

In [ ]:
x = 3
plt.close('all')
continuous_update = False
checkbox = {'Continuous': widgets.Checkbox()}
seed = np.random.normal(0, 0.1, [x])
layout = Layout(width='500px')
slider_settings = dict(continuous_update=continuous_update, layout=layout)
speeds = {str(i+1): widgets.FloatSlider(
              min = -2.0, 
              max = 2.0, 
              step = 0.01, 
              value = seed[i], **slider_settings) for i in range(x)}
fig = plt.figure()
ax = fig.add_subplot()
quality = {'Quality': widgets.FloatSlider(value=0.1, min=0, max=5, step=0.01, continuous_update=continuous_update)}
# widgets.FloatLogSlider
quality = {'Points': widgets.FloatLogSlider(value=100, base=10, min=1, max=6, step=0.1, **slider_settings)}
resolution = {'Resolution': widgets.IntSlider(value=100, min=1, max=1000, step=1, **slider_settings)}
interact_options = speeds | quality | checkbox | resolution
@interact(**interact_options)
def generate(c=R.cmaps, **interact_options):
    for widget in [quality]:
        widget[list(widget.keys())[0]].continuous_update = interact_options['Continuous']
    
#     global speeds
    speeds = np.array(list(interact_options.values())[:x])
    R = RouletteCurve(lengths=[1]*x, speeds=speeds)
    fig.canvas.draw()
#     cmap='Blues'
    render_options = dict(discard=True, clip=(0, 5), axis=ax, mode='hist', blending='add', hist_args=dict(bins=interact_options['Resolution']), cmap=c)
#     R.simulate(live_rendering=False, duration=interact_options['Quality'], render_each=2000, render_settings=render_options)

    R.simulate_accelerated(round(interact_options['Points']))
    print(len(R.points), R.speeds)
    R.render(**render_options)
    
# update vs setdata vs imshow vs... ?
#     plt.show()
# todo: simulate multiple tracks in parallel
# reuse existing results when changing resolution, colormap, etc.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='c', options=('inferno', 'plasma', 'rainbow', 'hot', 'cool', 'autum…

In [ ]:
# dict.values()
dict

In [ ]:
v = resolution['Resolution']

In [ ]:
v

IntSlider(value=100, continuous_update=False, description='Resolution', layout=Layout(width='500px'), max=1000…

In [ ]:
R.canvas.max()

6.0

In [ ]:
R.cmaps

['rainbow',
 'hot',
 'cool',
 'autumn',
 'winter',
 'summer',
 'Greys',
 'Purples',
 'Blues',
 'Greens',
 'Oranges',
 'Reds',
 'YlOrBr',
 'YlOrRd',
 'OrRd',
 'PuRd',
 'RdPu',
 'BuPu',
 'GnBu',
 'PuBu',
 'YlGnBu',
 'PuBuGn',
 'BuGn',
 'YlGn']

In [ ]:
# @interact(convolve, **dict(zip(map(str, range(1, 10)), [1]*9)))
x = 2
seed = np.random.normal(0, 0.1, [x**2])
seed[seed.size//2] = 1
kernel = {str(i+1): widgets.FloatSlider(
              min = -2.0, 
              max = 2.0, 
              step = 0.01, 
              value = seed[i]) for i in range(x**2)}

conv_settings = dict(mode='full', boundary='symm')
@interact(**kernel)
def convolve(**kernel):
    kernel = np.reshape(list(kernel.values()), [x, x])
    plt.imshow(signal.convolve2d(R.canvas, kernel, ), interpolation='none', cmap='Blues')
# convolve(kernel)
    
# iterated convolutions
# learning to produce dynamical system rules that fit some objective *
# approximate distribution of sample with smooth function?

interactive(children=(FloatSlider(value=0.04690160617608796, description='1', max=2.0, min=-2.0, step=0.01), F…

In [ ]:
# def denoise(pixels):

In [ ]:
R.canvas

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# hide

# random convolutions on noise
# apply neural network to each pixel/coordinate

# complex-valued dynamical systems

# todo: density approximation
# nesting spinners?
# 'pure' iterated functions *
# use ML to estimate histogram levels at each point
# stats
# ML-based denoising, point generation, etc.

# post-processing; e.g., scaling or squaring points
# AI upsampling/denoising
# compositing
# completion percentage
# chaos <-> order
# store points in compressed format
# track compute used
# monoids
# live editing/interaction
# real-time rendering
# interpolation
# simulatefor
# render-partial
# live averaging
# multidimensional color interpolation
# export LaTeX-formatted description of system
# todo: various optimizations
# todo: scatter plot rendering

In [ ]:
# np.stack(np.mgrid([np.arange(5.)]*2))

In [ ]:
# RouletteCurve.rotate(np.array([0, 5]), np.array([0, 1]), 1)

In [ ]:
theta = 2*math.pi*0.01
rMatrix = [
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta), np.cos(theta)]
]
rMatrix = np.array(rMatrix)
# rMatrix = np.array([rMatrix]*2).swapaxes(1,2)
print(rMatrix)
h = np.array([0, 0]).astype(float)
X = np.array([[0, 4], [0, 3]])[0].astype(float)
Y = np.array([0, 7]).astype(float)
Z = np.array([0, 10]).astype(float)
print(X.shape)
# ((X-h)[0] @ (rMatrix)[0])+h

xs, ys, zs = [], [], []
al2d = np.atleast_2d
def W(j, k):
    return ((rMatrix@(al2d(j).T-al2d(k).T))+al2d(k).T).T.squeeze()
for I in range(1001):
#     X=(X-h)@rMatrix+h
#     Y=(rMatrix@(Y-X))+X

    c = map(np.copy, [X, Y, Z])
    X_, Y_, Z_ = c
    X=W(X,h)
    Y=W(Y,X_)
#     Z=W(Z,Y_)
    
    xs+=[X]
    ys+=[Y]
#     zs+=[Z]
print(X)
print(Y)
print(Z)
q=2
plt.plot(np.transpose([np.arange(0,1001)]*q),np.array([xs, ys, zs][:q])[:,:,0].T)
# print(Z)
# loop aggregators

In [ ]:
# type(R.angles[0]) is float
isinstance(R.angles[0], float)

In [ ]:
# plt.plot(np.array(R.angles_)[:, 2])
# plt.plot(np.array(R.pivots_)[:,2,:].sum(axis=1))
# np.array(R.pivots_)[:5,1,0]
# R.points